WebScrapper for EWeb courses Canavan

In [1]:
#imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import sys
from selenium.webdriver.support.select import Select
import pandas as pd

In [2]:
#Term select input
x = input("Would you like to select the last Semester?: ")
def selTerm():
    if x != 'yes'|'no':
        print('Invalid Entry, please state "yes" or "no')
        selTerm()


In [3]:
#Driver for firefox and loading the Eweb url
driver = webdriver.Firefox()
driver.implicitly_wait(10)

driver.get("https://ssb-prod.ec.easternct.edu/PROD/bwskfcls.p_termsel")

In [4]:
# select term
elem = driver.find_elements(by = 'id', value = 'term_id')
if x == 'yes':
	s = Select(elem[0])
	s.select_by_index(2)

In [5]:
# submit term
elem= driver.find_elements(by = 'tag name', value = 'input')
elem[1].click()

In [6]:
#selecting both
both = driver.find_element(by = 'id', value = 'oc_id')
both.click()

In [7]:
#searching
btn = driver.find_element(by = 'name', value = 'SUB_BTN')
btn.click()

In [8]:
    # Locate all tables on the page
tbl = driver.find_elements("tag name", "table")


In [9]:
#This is where all the information will be stored and appened to for .csv exporting
info = []

for tbl_index, table in enumerate(tbl[2:], start=3):  # Enumerate
    
    # Locate the rows in the respective table
    rows = table.find_elements("tag name", "tr")
    
    # Loop through all rows starting with the 4th row 
    for row_index, row in enumerate(rows[3:], start=4):  # Enumerate from 4th row
        
        # Locate all the columns in this row
        col = row.find_elements("tag name", "td")
        
        # Using the columns that hold desired data(crse,sec,day,time,room,instrcutor)
        indices = [3,4, 5, 10, 11, 20, 21]
        data = []
        for col_index in indices:
            # Avoid error if column is not present
            if col_index < len(col):
                data.append(col[col_index].text)
            else:
                data.append("N/A")
        
        # Add the extracted data to info
        info.append(data)

# Creating of Pandas data frame
df = pd.DataFrame(info, columns=["Sub","Crse", "Sec", "Day", "Time", "Room", "Instructor"])

#Resolving blanks
for row in range(len(df)):  # Loop through row indices
    if df.iloc[row, 2] == ' ':  # Check if Sec has ' '
        df.iloc[row, 0] = df.iloc[row-1, 0]  # Copy values from the previous row
        df.iloc[row, 1] = df.iloc[row-1, 1] 
        df.iloc[row, 2] = df.iloc[row-1, 2]

# Export the DataFrame to a CSV file
df.to_csv("AllCourses.csv", index=False)

print("Data exported")


Data exported


In [10]:
#Close the webpage
driver.quit()

In [11]:
# Modify instructor names by removing all instances of '(P)'
df['Instructor'] = df['Instructor'].str.replace(r'\(P\)', "", regex=True).str.strip()


# Filter out rows where the instructor field contains invalid names
df = df[~df['Instructor'].isin(['TBA', '()', '',' ()', ')', '('])]

# Group by instructor and concatenate all unique times into a list
df_grouped = df.groupby('Instructor')['Time'].apply(lambda x: list(set(x))).reset_index()

# Export to a new CSV file
df_grouped.to_csv("InstructorSchedules.csv", index=False)

print("Instructor schedule exported successfully.")


Instructor schedule exported successfully.


In [12]:
import ast
def split_instructors(instructor_str):
    """
    Splits an instructor string into individual instructor names.
    If the string has more than one comma, we chunk them in pairs.
    Example:
      "Chirico, M (), Diller, J , Doyle, M" 
      -> ["Chirico, M ()", "Diller, J", "Doyle, M"]
    """
    # Split by commas
    parts = instructor_str.split(",")
    # Strip whitespace
    parts = [p.strip() for p in parts]
    
    # Pair them up (0/1, 2/3, 4/5, etc.)
    # If there's an odd part out at the end, we'll just keep it as is.
    instructors = []
    for i in range(0, len(parts), 2):
        if i + 1 < len(parts):
            combined = parts[i] + ", " + parts[i+1]
            instructors.append(combined.strip())
        else:
            # If there's no matching part, just add the last piece
            instructors.append(parts[i])
    
    return instructors

# 1. Read your CSV file
df = pd.read_csv("InstructorSchedules.csv")

# 2. Remove '(P)' from instructor names

# 3. Filter out invalid instructor names
invalid_names = ['TBA', '()', '', ' ()', ')', '(']
df = df[~df['Instructor'].isin(invalid_names)]

# 4. Parse the 'Time' column from string -> Python list
#    (Assumes each row looks like "['11:00 am-11:50 am', 'TBA']" in the CSV)
df['Time'] = df['Time'].apply(ast.literal_eval)

# 5. Remove "TBA" from the time lists
df['Time'] = df['Time'].apply(
    lambda times: [t for t in times if t.strip().upper() != 'TBA']
)

# 6. Split rows that have multiple instructors
expanded_rows = []
for _, row in df.iterrows():
    instructor_str = row['Instructor']
    time_list = row['Time']
    
    # Split the instructor field into one or more instructors
    instructors = split_instructors(instructor_str)
    
    # For each instructor, create a separate row with the same time list
    for inst in instructors:
        expanded_rows.append({
            'Instructor': inst,
            'Time': time_list
        })

# Create a new DataFrame from the expanded rows
df_expanded = pd.DataFrame(expanded_rows)

# 7. Export to a new CSV file
df_expanded.to_csv("InstructorSchedules_Cleaned.csv", index=False)

print("Instructor schedules have been cleaned and exported successfully.")

Instructor schedules have been cleaned and exported successfully.


In [13]:
# 1. Read the CSV file that has the Instructor and Time columns
df = pd.read_csv("InstructorSchedules_Cleaned.csv")

# 2. Remove the trailing " ()" from Instructor names
#    This uses a regex to remove any space + "()" at the end of the name.
df['Instructor'] = df['Instructor'].str.replace(r"\s*\(\)\s*$", "", regex=True).str.strip()

# 3. Convert the Time column from a string like "['09:30 am-10:45 am']" into a real Python list
df['Time'] = df['Time'].apply(ast.literal_eval)

# 4. Group by Instructor, then combine (concatenate) all Time lists into a single list
#    We'll also use a set to remove duplicates, if any. 
df_grouped = (
    df.groupby('Instructor')['Time']
      .apply(lambda series_of_lists: sorted({t for sublist in series_of_lists for t in sublist}))
      .reset_index()
)

# 5. Sort the grouped DataFrame alphabetically by Instructor
df_grouped = df_grouped.sort_values(by='Instructor').reset_index(drop=True)

# 6. Save the final result to a new CSV file
df_grouped.to_csv("InstructorGroupedTimes.csv", index=False)

print("Done! Instructors have been grouped, times combined, and the file is saved.")

Done! Instructors have been grouped, times combined, and the file is saved.


In [14]:


# 1. Read your CSV file (replace with your actual filename).
df = pd.read_csv("InstructorGroupedTimes.csv")

# 2. Convert the 'Time' column from a string representation of a list into an actual Python list.
#    This step is crucial because the CSV likely stores something like "['10:00 am-10:50 am', 'TBA']" as a string.
df['Time'] = df['Time'].apply(ast.literal_eval)

# 3. Remove 'TBA' from each list in the 'Time' column.
df['Time'] = df['Time'].apply(
    lambda times: [t for t in times if t.strip().upper() != 'TBA']
)

# 4. Export the cleaned-up data to a new CSV.
df.to_csv("InstructorClassSchedulesFinal.csv", index=False)

print("InstructorClassSchedulesFinal.csv has been created without TBA times.")

InstructorClassSchedulesFinal.csv has been created without TBA times.


GITHUB, BLANKS, Downloads, GITHUB REPOSITORY
DEVELOP WEB SCRAPER FOR OH